In [ ]:
import h5py
import numpy as np
import os
import pandas as pd
import pickle
import scipy
import scipy.interpolate
import unyt

In [ ]:
import matplotlib
import matplotlib.style
matplotlib.style.use( '/Users/zhafen/repos/clean-bold/clean-bold.mplstyle' )
import matplotlib.pyplot as plt
import matplotlib.gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import palettable

In [ ]:
import verdict
import trove

# Parameters

In [ ]:
sample_i = 0

In [ ]:
# Load parameters
pm = trove.link_params_to_config(
    '/Users/zhafen/repos/cgm_modeling_challenge/sample2.trove',
    script_id = 'nb.2',
)

In [ ]:
figure_dir = pm['figure_dir']
os.makedirs( figure_dir, exist_ok=True )

In [ ]:
prop_keys = [ 'l', 'T', 'nH', 'Z', ]

## Plotting

In [ ]:
logscale = {
    'l': True,
    'T': True,
    'nH': True,
    'Z': True,
}

In [ ]:
labels = {
    'l': r'$\ell$ [kpc]',
    'T': r'T [K]',
    'nH': r'$n_{\rm H}$ [cm$^{-3}$]',
    'Z': r'$Z$ [$Z_{\odot}$]',
}

In [ ]:
lims = {
    'T': [ 1e4, 1e6 ],
}

In [ ]:
panel_length = 4.

In [ ]:
found_color = palettable.cartocolors.qualitative.Safe_10.mpl_colors[1]
revised_color = palettable.cartocolors.qualitative.Safe_10.mpl_colors[0]

# Analysis

## Load Data

### Actual Data

In [ ]:
actual = verdict.Dict.from_hdf5( './data/synthetic_data/sample{}/theorists_file.h5'.format( sample_i ) )

In [ ]:
observers_data = verdict.Dict.from_hdf5( './data/synthetic_data/sample{}/observers_file.h5'.format( sample_i ) )

In [ ]:
provided = verdict.Dict.from_hdf5( './data/synthetic_data_samples/sample0.h5' )

### Modeled

#### Sameer & Charlton

In [ ]:
modeling_group = 'sameer_charlton'

##### Original Results

In [ ]:
modeled_raw = verdict.Dict.from_hdf5( './data/modeling_results/{}/sample{}/sample{}.hdf5'.format( modeling_group, sample_i, sample_i ) )

In [ ]:
derived = pd.read_csv( './data/modeling_results/sameer_charlton/sample{}/derived.txt'.format( sample_i ), sep='\t', index_col=0 )

In [ ]:
modeled = {
    'metallicity': np.array([ modeled_raw[str(i+1)][0] for i in range( actual['metallicity'].size ) ]),
    'emetallicity': np.array([ modeled_raw[str(i+1)][1] for i in range( actual['metallicity'].size ) ]),
    'length': derived.values[:,0],
    'H_column': derived.values[:,1],
    'temperature': derived.values[:,2],
    'H_density': derived.values[:,3],
}

In [ ]:
fp = './data/modeling_results/{}/sample{}/firstiteration.pickle'.format( modeling_group, sample_i )
columns = dict( pd.read_pickle( fp ) )

In [ ]:
for key, item in columns.items():
    item_arr = np.array(item)
    modeled['{}'.format( key )] = item_arr[:,0]
    modeled['e{}'.format( key )] = item_arr[:,1]

In [ ]:
modeled = pd.DataFrame( modeled )
modeled = modeled.set_index( np.arange( modeled.shape[0] ) + 1 )

In [ ]:
modeled

##### Revised

In [ ]:
revised = pd.read_csv( './data/modeling_results/{}/sample{}/revised_params.csv'.format( modeling_group, sample_i ) )
revised = revised.set_index( np.arange( revised.shape[0] ) + 1 )

In [ ]:
fp = './data/modeling_results/{}/sample{}/seconditeration.pickle'.format( modeling_group, sample_i )
columns = dict( pd.read_pickle( fp ) )

In [ ]:
for key, item in columns.items():
    item_arr = np.array(item)
    revised['{}'.format( key )] = item_arr[:,0]
    revised['e{}'.format( key )] = item_arr[:,1]

In [ ]:
revised

#### Mohapatra

In [ ]:
met_mohapatra = np.full( ( actual['metallicity'].size ),  np.nan,  )
emet_mohapatra = np.full( ( actual['metallicity'].size ),  np.nan,  )
met_mohapatra[7] = -0.52
emet_mohapatra[7] = 0.41
met_mohapatra[9] = 0.48
emet_mohapatra[9] = 0.39

## Data for Interpretation

In [ ]:
phase_diagram = verdict.Dict.from_hdf5( './data/m12i_phase_diagram_z0.25.h5' )

### Address Possible Scrambling of Sightline Inds

#### Best case match closest metallicities

In [ ]:
model_sort_inds = np.argsort( modeled['metallicity'] )
actual_sort_inds = np.argsort( actual['metallicity'] )

#### Use Sameer's columns to assess mixup

In [ ]:
columns = pd.read_csv( './data/modeling_results/sameer_charlton/sample{}/columns.txt'.format( sample_i ), sep='\t', index_col=0 )

In [ ]:
columns.loc['HI']

In [ ]:
provided['H I']['logN']

In [ ]:
np.log10( actual['H I'] )

It doesn't look like there was a mixup...

# Results

## 1-to-1 comparison

In [ ]:
colormap = palettable.cartocolors.qualitative.Safe_10.mpl_colors

In [ ]:
fig = plt.figure( figsize=(8,8), facecolor='w' )
ax = plt.gca()

ax.errorbar(
    np.log10( actual['metallicity'] ),
    modeled['metallicity'],
    yerr = modeled['emetallicity'],
    marker = 'o',
    markersize = 10,
    color = colormap[0],
    linestyle = 'none',
    label = 'Sameer & Charlton',
)
for i, met in enumerate( np.log10( actual['metallicity'] ) ):
    ax.annotate(
        text = '{}'.format( i + 1 ),
        xy = ( met, modeled['metallicity'].loc[i + 1] ),
        xytext = ( -5, 5 ),
        textcoords = 'offset points',
        fontsize = 14,
        va = 'bottom',
        ha = 'right',
    )

ax.errorbar(
    np.log10( actual['metallicity'] ),
    revised['Z'],
    yerr = revised['errZ'],
    marker = 'X',
    markersize = 10,
    color = colormap[1],
    linestyle = 'none',
    label = 'Sameer & Charlton - Revised',
)

# ax.errorbar(
#     np.log10( actual['metallicity'] ),
#     met_mohapatra,
#     yerr = emet_mohapatra,
#     marker = 's',
#     markersize = 10,
#     color = colormap[2],
#     linestyle = 'none',
#     label = 'Mohapatra',
# )
# ax.plot(
#     [ np.log10( actual['metallicity'] )[8], ]*2,
#     [ 0.54, 1000 ],
#     color = colormap[2],
# )

# ax.errorbar(
#     np.log10( actual['metallicity'] )[actual_sort_inds],
#     modeled['metallicity'][model_sort_inds],
#     yerr = modeled['emetallicity'][model_sort_inds],
#     marker = 'o',
#     markersize = 10,
#     color = '0.5',
#     linestyle = 'none',
#     zorder = -5,
# )

ax.plot(
    [ -2.2, 1 ],
    [ -2.2, 1 ],
    color = '0.5',
    linestyle = '--',
    linewidth = 3,
    zorder = -10,
)

ax.tick_params( length=10, width=1.5, labelsize=18 )

ax.set_xlabel( r'$\log_{10}( Z_{\rm actual}/Z_\odot )$', fontsize=22, )
ax.set_ylabel( r'$\log_{10}( Z_{\rm modeled}/Z_\odot )$', fontsize=22, )

ax.set_xlim( -2.2, 1 )
ax.set_ylim( -2.2, 1 )

ax.set_aspect( 'equal' )

l = ax.legend(
    prop={'size': 14 },
)

In [ ]:
fig = plt.figure( figsize=(8,8), facecolor='w' )
ax = plt.gca()

ax.scatter(
    np.log10( actual['H_density'] ),
    modeled['H_density'],
    marker = 'o',
    s = 100,
    color = 'k',
)

# ax.errorbar(
#     np.log10( actual['metallicity'] )[actual_sort_inds],
#     modeled['metallicity'][model_sort_inds],
#     yerr = modeled['emetallicity'][model_sort_inds],
#     marker = 'o',
#     markersize = 10,
#     color = '0.5',
#     linestyle = 'none',
#     zorder = -5,
# )

bounds = [ -4.5, 0 ]

ax.plot(
    bounds,
    bounds,
    color = '0.5',
    linestyle = '--',
    linewidth = 3,
    zorder = -10,
)

ax.tick_params( length=10, width=1.5, labelsize=18 )

ax.set_xlabel( r'$\log_{10}( n_{\rm H,actual}/{\rm cm^{-3}} )$', fontsize=22, )
ax.set_ylabel( r'$\log_{10}( n_{\rm H,modeled}/{\rm cm^{-3}} )$', fontsize=22, )

ax.set_xlim( bounds )
ax.set_ylim( bounds )

ax.set_aspect( 'equal' )

# plt.savefig( './figures/sample0/nH_modeled_vs_actual.pdf', bbox_inches='tight' )

In [ ]:
fig = plt.figure( figsize=(8,8), facecolor='w' )
ax = plt.gca()

ax.scatter(
    np.log10( actual['temperature'] ),
    modeled['temperature'],
    marker = 'o',
    s = 100,
    color = 'k',
)

# ax.errorbar(
#     np.log10( actual['metallicity'] )[actual_sort_inds],
#     modeled['metallicity'][model_sort_inds],
#     yerr = modeled['emetallicity'][model_sort_inds],
#     marker = 'o',
#     markersize = 10,
#     color = '0.5',
#     linestyle = 'none',
#     zorder = -5,
# )

bounds = [ 4, 6 ]

ax.plot(
    bounds,
    bounds,
    color = '0.5',
    linestyle = '--',
    linewidth = 3,
    zorder = -10,
)

ax.tick_params( length=10, width=1.5, labelsize=18 )

ax.set_xlabel( r'$\log_{10}( T_{\rm actual}/{\rm K} )$', fontsize=22, )
ax.set_ylabel( r'$\log_{10}( T_{\rm modeled}/{\rm K} )$', fontsize=22, )

ax.set_xlim( bounds )
ax.set_ylim( bounds )

ax.set_aspect( 'equal' )

# plt.savefig( './figures/sample0/T_modeled_vs_actual.pdf', bbox_inches='tight' )

In [ ]:
fig = plt.figure( figsize=(8,8), facecolor='w' )
ax = plt.gca()

ax.scatter(
    np.log10( actual['length'] ),
    np.log10( modeled['length'] ),
    marker = 'o',
    s = 100,
    color = 'k',
)

# ax.errorbar(
#     np.log10( actual['metallicity'] )[actual_sort_inds],
#     modeled['metallicity'][model_sort_inds],
#     yerr = modeled['emetallicity'][model_sort_inds],
#     marker = 'o',
#     markersize = 10,
#     color = '0.5',
#     linestyle = 'none',
#     zorder = -5,
# )

bounds = [ -1, 6 ]

ax.plot(
    bounds,
    bounds,
    color = '0.5',
    linestyle = '--',
    linewidth = 3,
    zorder = -10,
)

ax.tick_params( length=10, width=1.5, labelsize=18 )

ax.set_xlabel( r'$\log_{10}( \ell_{\rm actual}/{\rm kpc} )$', fontsize=22, )
ax.set_ylabel( r'$\log_{10}( \ell_{\rm modeled}/{\rm kpc} )$', fontsize=22, )

ax.set_xlim( bounds )
ax.set_ylim( bounds )

ax.set_aspect( 'equal' )

# plt.savefig( './figures/sample0/l_modeled_vs_actual.pdf', bbox_inches='tight' )

## Sightlines in Context of Priors

### Interpolate to Get PDF Values

In [ ]:
dlogT = phase_diagram['logT_edges'][1] - phase_diagram['logT_edges'][0]
dlogHDen = phase_diagram['logH_density_edges'][1] - phase_diagram['logH_density_edges'][0]
pdf = phase_diagram['hist'] / ( dlogT * dlogHDen )

In [ ]:
logHDen_centers = phase_diagram['logH_density_edges'][:-1] + 0.5 * dlogHDen
logT_centers = phase_diagram['logT_edges'][:-1] + 0.5 * dlogT

In [ ]:
interp_fn = scipy.interpolate.RectBivariateSpline(
    logHDen_centers,
    logT_centers,
    pdf,
)

In [ ]:
pdf_values = np.array([ interp_fn( np.log10( actual['H_density'][i] ), np.log10( actual['temperature'][i] ) )[0][0] for i in range( actual['metallicity'].size ) ])

In [ ]:
pdf_values[pdf_values<0] = pdf[np.nonzero(pdf)].min()

In [ ]:
fig = plt.figure( figsize=(15,10), facecolor='w' )
ax = plt.gca()

img = ax.imshow(
    np.rot90( pdf ),
    extent = [ logHDen_centers[0], logHDen_centers[-1], logT_centers[0], logT_centers[-1] ],
    cmap = palettable.cubehelix.classic_16_r.mpl_colormap,
    norm = matplotlib.colors.LogNorm(),
)

vmin = np.nanmin( pdf[np.nonzero(pdf)] )
vmax = np.nanmax( pdf[np.nonzero(pdf)] )
ax.scatter(
    np.log10( actual['H_density'] ),
    np.log10( actual['temperature'] ),
    edgecolor = 'k',
    s = 100,
    c = pdf_values,
    cmap = palettable.cubehelix.classic_16_r.mpl_colormap,
    norm = matplotlib.colors.LogNorm( vmin, vmax ),
)

for i in range( actual['H_density'].size ):
    ax.annotate(
        text = str( i + 1 ),
        xy = np.log10([ actual['H_density'][i], actual['temperature'][i] ]),
        fontsize = 20,
        color = 'k',
        xytext = ( 0, 3.5 ),
        textcoords = 'offset points',
        ha = 'center',
        va = 'bottom',
    )

# Colorbar
# Create divider for existing axes instance
divider = make_axes_locatable(ax)
# Append axes to the right of ax, with 5% width of ax
cax = divider.append_axes("right", pad=0.05, size='5%')
cb = plt.colorbar( img, cax=cax )
cb.ax.tick_params(which='major', labelsize=18, length=10, width=1.5 )
cb.ax.tick_params(which='minor', labelsize=18, length=5, width=1. )
ax.annotate(
    text = 'PDF',
    xy = ( 1, 1 ),
    xytext = ( 0, 10 ),
    xycoords = 'axes fraction',
    textcoords = 'offset points',
    fontsize = 24,
)


ax.tick_params( length=10, width=1.5, labelsize=18 )

ax.set_xlabel( r'$\log_{10} n_{\rm H}$ (cm$^{-3}$)', fontsize=22 )
ax.set_ylabel( r'$\log_{10} T$ (K)', fontsize=22 )

ax.set_aspect( 'equal' )

savefile = os.path.join( figure_dir, 'phase_space.png' )
print( 'Saving at {}'.format( savefile ) )
plt.savefig( savefile, bbox_inches='tight' )

## Comparison Including Expected Frequency

In [ ]:
y_labels = {
    'metallicity': r'$\log_{10} ( Z_{\rm modeled}/Z_{\rm actual} )$',
    'H_density': r'$\log_{10} ( n_{\rm H, modeled}/n_{\rm H, actual} )$',
    'temperature': r'$\log_{10} ( T_{\rm modeled}/T_{\rm actual} )$',
    'length': r'$\log_{10} (\ell_{\rm modeled} / \ell_{\rm actual} )$',
}

### Properties with Ratios

In [ ]:
vmin = np.nanmin( pdf[np.nonzero(pdf)] )
vmax = np.nanmax( pdf[np.nonzero(pdf)] )

In [ ]:
for key in [ 'metallicity', 'H_density', 'temperature', 'length' ]:

    fig = plt.figure( figsize=(8,8), facecolor='w' )
    ax = plt.gca()

    if key != 'length':
        ys = modeled[key] - np.log10( actual[key] )
    else:
        ys = np.log10( modeled[key] / actual[key] )
        
    ax.scatter(
        np.arange( actual[key].size ) + 1,
        ys,
        edgecolor = 'k',
        s = 100,
        c = pdf_values,
        cmap = palettable.cubehelix.classic_16_r.mpl_colormap,
        norm = matplotlib.colors.LogNorm( vmin, vmax ),
    )

    if key == 'metallicity':
        
        ax.errorbar(
            np.arange( actual[key].size ) + 1,
            modeled[key] - np.log10( actual[key] ),
            yerr = modeled['emetallicity'],
            linestyle = 'none',
            color = 'k',
            zorder = -10,
        )
        
        ax.scatter(
            np.arange( actual[key].size ) + 1,
            revised['Z'] - np.log10( actual[key] ),
            s = 100,
            color = colormap[1],
            zorder = -9,
        )
        ax.errorbar(
            np.arange( actual[key].size ) + 1,
            revised['Z'] - np.log10( actual[key] ),
            yerr = revised['errZ'],
            linestyle = 'none',
            color = colormap[1],
            zorder = -10,
        )
        
#         # Mohapatra results
#         ax.errorbar(
#             np.arange( actual[key].size ) + 1,
#             met_mohapatra,
#             yerr = emet_mohapatra,
#             marker = 's',
#             markersize = 10,
#             color = colormap[1],
#             linestyle = 'none',
#             label = 'Mohapatra',
#             zorder = -10
#         )
#         ax.plot(
#             [ 9, ]*2,
#             [ 0.54, 1000 ],
#             color = colormap[1],
#         )

    ax.axhline(
        0,
        color = '0.5',
        linestyle = '--',
        linewidth = 3,
        zorder = -10,
    )

    plt.xticks( np.arange( actual['metallicity'].size ) + 1, )
    ax.tick_params( length=10, width=1.5, labelsize=18 )

    ax.set_xlabel( r'Sightline ID', fontsize=22 )
    ax.set_ylabel( y_labels[key], fontsize=22 )
    
    if key == 'length':
        ax.set_ylim( -1.5, 4 )
    elif key == 'metallicity':
        ax.set_ylim( -1.5, 1.5 )
    else:
        ax.set_ylim( -4, 1.5 )
        
#     plt.savefig( './figures/sample0/comparison_{}_inc_freq.pdf'.format( key ), bbox_inches='tight' )

### Properties without Ratios

In [ ]:
found = verdict.Dict({
    'l': modeled['length'].values * unyt.kpc,
    'Z': 10.**modeled['metallicity'].values * unyt.Zsun,
    'T': 10.**modeled['temperature'].values * unyt.K,
    'nH': 10.**modeled['H_density'].values * unyt.cm**-3,
})

In [ ]:
found_revised = verdict.Dict({
    'Z': 10.**revised['Z'].values * unyt.Zsun,
    'T': 10.**revised['T'].values * unyt.K,
    'nH': 10.**revised['ne'].values * unyt.cm**-3,
})

In [ ]:
actual['l'] = actual['length'] * unyt.kpc
actual['Z'] = actual['metallicity'] * unyt.Zsun
actual['T'] = actual['temperature'] * unyt.K
actual['nH'] = actual['H_density'] * unyt.cm**-3
actual = verdict.Dict( actual )

In [ ]:
xs = modeled.index

In [ ]:
clean_mosaic = [
    [ 'T', 'l', ],
    [ 'nH', 'Z', ],
]

In [ ]:
# Setup Figure
n_rows_clean = 2
n_cols_clean = 2
aspect_ratio = 2
fig = plt.figure( figsize=(n_cols_clean*panel_length*aspect_ratio, n_rows_clean*panel_length), facecolor='w' )
ax_dict = fig.subplot_mosaic(
    clean_mosaic,
)

for key in prop_keys:
    
    ax = ax_dict[key]
    
    ratio = np.log10( actual[key] / found[key] )
    ordered_inds = np.argsort( np.abs( ratio ) )
    
    # Shade regions
    norm = matplotlib.colors.LogNorm( vmin, vmax )
    for i, c_value in enumerate( pdf_values[ordered_inds] ):
        
        c = palettable.cubehelix.classic_16_r.mpl_colormap( norm( c_value ) )
        
        ax.fill_between(
            [ xs[i] - 0.5, xs[i] + 0.5 ],
            [ 0, 0 ],
            [ 1, 1 ],
            transform = matplotlib.transforms.blended_transform_factory( ax.transData, ax.transAxes ),
            color = c,
        )
    
    # Actual
    ax.scatter(
        xs,
        actual[key][ordered_inds],
        s = 100,
        color = 'k',
#         edgecolor = 'k',
#         c = pdf_values[ordered_inds],
#         cmap = palettable.cubehelix.classic_16_r.mpl_colormap,
#         norm = matplotlib.colors.LogNorm( vmin, vmax ),
        zorder = 100,
    )
    
    # Original
    ax.scatter(
        xs,
        found[key][ordered_inds],
        color = found_color,
        s = 150,
        zorder = 50,
    )
    
    # Revised
    try:
        ax.scatter(
            xs,
            found_revised[key][ordered_inds],
            color = revised_color,
            s = 200,
        )
    except KeyError:
        pass
    
    if logscale[key]:
        ax.set_yscale( 'log' )
        
    # X ticks
    ax.set_xticks( xs )
    ax.set_xticklabels( ordered_inds )
    
    ax.set_xlim( xs[0] - 0.5, xs[-1] + 0.5 )
    if key in lims:
        ax.set_ylim( lims[key] )
    
# Cleanup
for x_key, ax in ax_dict.items():
    
    if x_key == 'legend':
        continue
    
    subplotspec = ax.get_subplotspec()
        
    ax.set_ylabel( labels[x_key], fontsize=16 )
    if subplotspec.is_last_row():
        ax.set_xlabel( 'sightline ID', fontsize=16 )
        
savefile = os.path.join( figure_dir, 'comparison.pdf' )
print( 'Saving at {}'.format( savefile ) )
plt.savefig( savefile, bbox_inches='tight' )

## Corner Plot Comparison

### Plot

In [ ]:
mosaic = [
    [ 'l', 'legend', '.', '.' ],
    [ 'T_l', 'T', '.', '.' ],
    [ 'nH_l', 'nH_T', 'nH', '.' ],
    [ 'Z_l', 'Z_T', 'Z_nH', 'Z', ],
]

In [ ]:
# Setup Figure
n_cols = len( prop_keys )
fig = plt.figure( figsize=( panel_length*n_cols, panel_length*n_cols ), facecolor='w' )
ax_dict = fig.subplot_mosaic(
    mosaic,
)

# Loop through all properties
for j, x_key in enumerate( prop_keys ):
    for k, y_key in enumerate( prop_keys ):

        # Avoid duplicates
        if k < j:
            continue 
            
        # Single property comparison
        if j == k:
            ax = ax_dict[x_key]
            subplotspec = ax.get_subplotspec()
            
            x_label = labels[x_key]
            y_label = 'sightline ID'
                        
        # 2D comparisons
        else:
            try:
                ax = ax_dict['{}_{}'.format( x_key, y_key )]
            except KeyError:
                ax = ax_dict['{}_{}'.format( y_key, x_key )]
            subplotspec = ax.get_subplotspec()
            
            # Actual values
            ax.scatter(
                actual[x_key],
                actual[y_key],
                color = 'k',
            )
            
            # Modeled values
            ax.scatter(
                found[x_key],
                found[y_key],
                color = found_color,
            )
            
            # Revised modeled values
            try:
                ax.scatter(
                    found_revised[x_key],
                    found_revised[y_key],
                    color = revised_color,
                )
            except KeyError:
                pass
            
            if logscale[x_key]:
                ax.set_xscale( 'log' )
            if logscale[y_key]:
                ax.set_yscale( 'log' )
                
            x_label = labels[x_key]
            y_label = labels[y_key]

        if subplotspec.is_last_row():
            ax.set_xlabel( x_label, fontsize=16 )
        if subplotspec.is_first_col():
            ax.set_ylabel( y_label, fontsize=16 )

## How Much Column Density Agreement is Enough?

In [ ]:
inds = np.array([ 5, 7, 8, 9 ])

In [ ]:
ions = [ 'H I', 'Si II', 'Si III', 'Si IV', 'N II', 'N III', 'N V', 'C II', 'C III', 'O I', 'O VI', ]

In [ ]:
n_rows = 3
n_cols = 4
fig = plt.figure( figsize=(n_cols*5,n_rows*4), facecolor = 'w' )
ax = plt.gca()

gs = matplotlib.gridspec.GridSpec( n_rows, n_cols )

gs.update( hspace=0.001, wspace=0.001 )

i = 0
j = 0
for k, ion in enumerate( ions ):
    
    ax_ij = fig.add_subplot( gs[j,i] )
    
    ax_ij.scatter(
        np.arange( inds.size ),
        modeled[ion.replace( ' ', '' )][inds+1] - provided[ion]['logN'][inds],
        color = found_color,
        s = 200,
        label = 'original',
    )
    
    ax_ij.errorbar(
        np.arange( inds.size ),
        modeled[ion.replace( ' ', '' )][inds+1] - provided[ion]['logN'][inds],
        yerr = provided[ion]['elogN'][inds],
#         yerr = modeled['e' + ion.replace( ' ', '' )][inds],
        linestyle = 'none',
        color = found_color,
        zorder = -10,
        linewidth = 4,
    )
    
    ax_ij.scatter(
        np.arange( inds.size ),
        revised[ion.replace( ' ', '' )][inds+1] - provided[ion]['logN'][inds],
        s = 200,
        color = revised_color,
        zorder = -5,
        label = 'revised',
    )
    ax_ij.errorbar(
        np.arange( inds.size ),
        revised[ion.replace( ' ', '' )][inds+1] - provided[ion]['logN'][inds],
        yerr = provided[ion]['elogN'][inds],
#         yerr = revised['e' + ion.replace( ' ', '' )][inds],
        linestyle = 'none',
        color = revised_color,
        zorder = -10,
        linewidth = 4,
    )
    
#     ax_ij.scatter(
#         inds + 1,
#         np.log10( actual[ion][inds] ),
#         color = 'k',
#     )

    ax_ij.axhline(
        0,
        color = '0.5',
        linestyle = '--',
        linewidth = 3,
        zorder = -10,
    )
    
    ax_ij.annotate(
        text = ion,
        xy = (1, 1),
        xycoords = 'axes fraction',
        xytext = ( -5, -5 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
    )
    
    ax_ij.set_ylim( -1.5, 2.5 )
    
    # Adjust ticks
    plt.xticks( np.arange( inds.size ), inds + 1 )
    ax_ij.tick_params( labelsize=15 )
    if i != 0:
        ax_ij.tick_params( left=False, labelleft=False )
    
    # Move to next axis in line
    if i > n_rows - 1:
        j += 1
        i = 0
    else:
        i += 1
        
        
handles = [
    matplotlib.lines.Line2D([0], [0], marker='o', color=found_color, label='original', markersize=15),
    matplotlib.lines.Line2D([0], [0], marker='o', color=revised_color, label='revised', markersize=15),
]

ax.legend(
    handles = handles,
    loc = 'lower right',
    prop = { 'size': 22 },
)

# Removing spines and tick marks
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params( bottom=False, left=False, labelleft=False, labelbottom=False )

ax.set_xlabel( 'Sightline ID', fontsize=26, labelpad=30 )
ax.set_ylabel( r'$\log_{10}( N_{\rm modeled} / N_{\rm actual} )$', fontsize=26, labelpad=30 )

savefile = os.path.join( figure_dir, 'column_den.pdf' )
print( 'Saving at {}'.format( savefile ) )
plt.savefig( savefile, bbox_inches='tight' )

## Noising of Data

In [ ]:
ions = list( provided.keys() )

In [ ]:

for ion in ions:
    fig = plt.figure( figsize=(8,8), facecolor='w' )
    ax = plt.gca()

    xs = np.log10( actual[ion] )
    ys = provided[ion]['logN']
    eys = provided[ion]['elogN']

    ax.errorbar(
        xs,
        ys,
        yerr = eys,
        marker = 'o',
        markersize = 10,
        color = 'k',
        linestyle = 'none',
    )

    bounds = [
        min( np.nanmin( xs[np.isfinite(xs)] ), np.nanmin( ys[np.isfinite(ys)] ) ) - 0.5,
        max( np.nanmax( xs[np.isfinite(xs)] ), np.nanmax( ys[np.isfinite(ys)] ) ) + 0.5,
    ]
    ax.plot(
        bounds,
        bounds,
        color = '0.5',
        linestyle = '--',
        linewidth = 3,
        zorder = -10,
    )
    
    ax.annotate(
        text = ion,
        xy = ( 0, 1 ),
        xycoords = 'axes fraction',
        xytext = ( 5, -5 ),
        textcoords = 'offset points',
        fontsize = 22,
        ha = 'left',
        va = 'top',
    )
    
    ax.tick_params( length=10, width=1.5, labelsize=18 )

    ax.set_xlabel( r'$\log_{10}( N_{\rm ion,\,actual} )$', fontsize=22, )
    ax.set_ylabel( r'$\log_{10}( N_{\rm ion,\,provided} )$', fontsize=22, )

    ax.set_xlim( bounds )
    ax.set_ylim( bounds )

    ax.set_aspect( 'equal' )